Pick all rows where branch id is not null and which are not merged (merge_flag = false)
Remove all leading and trailing spaces in branch country and convert to upper case

In [0]:
df = spark.sql("SELECT branch_id, branch_city, upper(trim(branch_country)) as branch_country FROM bronze.branch WHERE branch_id IS NOT NULL AND merge_flag = FALSE")


Merge to silver layer

In [0]:
df.createOrReplaceTempView("transformed_branch")
spark.sql("MERGE INTO silver.branch AS T USING transformed_branch as S ON T.branch_id = S.branch_id WHEN MATCHED THEN UPDATE SET T.branch_city=S.branch_city, T.branch_country=S.branch_country, T.merge_timestamp=current_timestamp() WHEN NOT MATCHED THEN INSERT (branch_id, branch_city, branch_country, merge_timestamp) VALUES (S.branch_id, S.branch_city, S.branch_country, current_timestamp())")

update merge_flag in bronze to true

In [0]:
spark.sql("update bronze.branch set merge_flag = true where merge_flag = false")